In [36]:
# experiment with 0.2m per pixel IGN raster (good qual) between 400, 800 and 1200 pixel
# doesn't perform very well
PATCH_SIZE=1200
OVERLAP=0.2

GEOTIFF_PATH = "../dataset_marseille/satellite_raster_sample_centre_IGN_0.2m-resolution.tif"
OUTPUT_PATH = "../dataset_marseille/deepforest_prediction_IGN_0.2m_{}px_{}overlap.shp".format(PATCH_SIZE, OVERLAP)

In [ ]:
# experiment with 0.1m per pixel IGN raster (better qual) with 400 px
PATCH_SIZE=400
OVERLAP=0.15

GEOTIFF_PATH = "../dataset_marseille/satellite_raster_sample_centre_IGN_0.1m-resolution.tif"
OUTPUT_PATH = "../dataset_marseille/deepforest_prediction_IGN_0.1m_{}px_{}overlap.shp".format(PATCH_SIZE, OVERLAP)

In [40]:
# experiment with 0.1m per pixel Mapbox raster (not great qual) with 400 px
PATCH_SIZE=400
OVERLAP=0.15

GEOTIFF_PATH = "../dataset_marseille/satellite_raster_sample_centre_mapbox_0.1m-resolution.tif"
OUTPUT_PATH = "../dataset_marseille/deepforest_prediction_mapbox_0.1m_{}px_{}overlap.shp".format(PATCH_SIZE, OVERLAP)

In [1]:
from deepforest import deepforest
from deepforest import get_data

#Geospatial packages

import shapely
import geopandas
import rasterio

Using TensorFlow backend.


In [3]:
test_model = deepforest.deepforest()
test_model.use_release()


Reading config file: C:\Users\matth\scoop\apps\miniconda3\current\envs\deepforest\lib\site-packages\deepforest\data\deepforest_config.yml
A blank deepforest object created. To perform prediction, either train or load an existing model.
Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/v0.3.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/v0.3.0

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [42]:
# remove exception from PIL Image for large files
# by setting the MAX to None
from PIL import Image
Image.MAX_IMAGE_PIXELS = None


raster_path = GEOTIFF_PATH

boxes = test_model.predict_tile(
    raster_path,
    # return_plot draws the image
    return_plot = False,
    patch_size=PATCH_SIZE,
    patch_overlap=OVERLAP)

100%|████████████████████████████████████████████████████████████████████████████| 6099/6099 [3:02:07<00:00,  1.79s/it]


2487 predictions in overlapping windows, applying non-max supression
2192 predictions kept after non-max suppression


In [31]:
# Write shapefile with prediction

with rasterio.open(raster_path) as dataset:
    bounds = dataset.bounds
    pixelSizeX, pixelSizeY  = dataset.res

#subtract origin. Recall that numpy origin is top left! Not bottom left.
boxes["xmin"] = (boxes["xmin"] *pixelSizeX) + bounds.left
boxes["xmax"] = (boxes["xmax"] * pixelSizeX) + bounds.left
boxes["ymin"] = bounds.top - (boxes["ymin"] * pixelSizeY)
boxes["ymax"] = bounds.top - (boxes["ymax"] * pixelSizeY)

# combine column to a shapely Box() object, save shapefile
boxes['geometry'] = boxes.apply(lambda x: shapely.geometry.box(x.xmin,x.ymin,x.xmax,x.ymax), axis=1)
boxes = geopandas.GeoDataFrame(boxes, geometry='geometry')

#set same epsg as source data
boxes.crs = {'init' :'epsg:3857'}

# from https://epsg.io/3857
prj = '''PROJCS["WGS 84 / Pseudo-Mercator",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Mercator_1SP"],
    PARAMETER["central_meridian",0],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["X",EAST],
    AXIS["Y",NORTH],
    EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs"],
    AUTHORITY["EPSG","3857"]]'''

boxes.to_file(OUTPUT_PATH, driver='ESRI Shapefile',crs=prj)
